In [36]:
def read(filename):
    with open(filename, 'r') as f:
        N, K = map(int, f.readline().split())
        d = [0] + list(map(float, f.readline().split()))
        t = np.zeros((N + 1, N + 1))
        for i in range(N + 1):
            t[i] = list(map(float, f.readline().split()))

    return N, K, d, t


import numpy as np

def cal_cost(router, t, d):
    cost = [0 for i in range(len(router))]
    for i in range(len(router)):
        for j in range(len(router[i]) - 1):
            cost[i] += t[router[i][j]][router[i][j + 1]] + d[router[i][j + 1]]
    return cost

def UCV(visited):
        unvisited = []
        for pos in range(len(visited)):
            if visited[pos] == 0:
                unvisited.append(pos)
        return unvisited

def local_search_greedy1(file_data):
    N, K, d, t = read(file_data)
    router = []
    for i in range(K):
        router.append([0])

    visited = np.zeros(N + 1, dtype = int)
    visited[0] = 1      
        
    k = 0
    while sum(visited) != N + 1:
        start = router[k][-1]
        next_pos = 0
        next_time = 1000000
        unvisited = UCV(visited)
        for pos in unvisited:
            if t[start][pos] + d[pos] < next_time:
                next_time = t[start][pos] + d[pos]
                next_pos = pos
        router[k].append(next_pos)
        visited[next_pos] = 1
        k = (k + 1) % K
    
    for i in range(K):
        router[i].append(0)
    cost = [0 for i in range(K)]
    for i in range(K):
        for j in range(len(router[i]) - 1):
            cost[i] += t[router[i][j]][router[i][j + 1]] + d[router[i][j + 1]]
    history_count = 0
    while True:
        max_cost = max(cost)
        max_index = [i for i, j in enumerate(cost) if j == max_cost]
        router_neighborhood = []

        swap1 = np.random.choice(max_index)
        swap2 = np.random.randint(0, K)
        while swap2 == swap1:
            swap2 = np.random.randint(0, K)
        
        if len(router[swap1]) == 3:
            a1 = 1
            b1 = 2
        else:
        #a1 != b1
            a1 = np.random.randint(1, len(router[swap1]) - 2)
            b1 = np.random.randint(a1 + 1, len(router[swap1]) - 1)

        if len(router[swap2]) == 3:
            a2 = 1
            b2 = 2
        else:
            a2 = np.random.randint(1, len(router[swap2]) - 2)
            b2 = np.random.randint(a2 + 1, len(router[swap2]) - 1)

        new_router1 = router[swap1][:a1] + router[swap2][a2:b2] + router[swap1][b1:]
        new_router2 = router[swap2][:a2] + router[swap1][a1:b1] + router[swap2][b2:]

        for i in range(K):
            if i == swap1:
                router_neighborhood.append(new_router1)
            elif i == swap2:
                router_neighborhood.append(new_router2)
            else:
                router_neighborhood.append(router[i])
        
        cost_neighborhood = cal_cost(router_neighborhood, t, d)
        new_max_cost = max(cost_neighborhood)
        if new_max_cost < max_cost:
            router = router_neighborhood
            cost = cost_neighborhood
            history_count = 0
        else:
            history_count += 1
        if history_count == 5000:
            break
        

    print(K)
    for sub in router:
        print(len(sub))
        for pos in sub:
            print(pos, end = ' ')
        print()
    return router, cost, K


In [37]:

import time

In [38]:
data = ["data_5_2.txt", "data_5_3.txt", "data_10_2.txt", "data_10_3.txt","data_10_5.txt", "data_15_3.txt", "data_15_5.txt", "data_15_7.txt", "data_20_5.txt", "data_20_7.txt","data_20_10.txt", "data_50_5.txt", "data_50_10.txt", "data_50_20.txt", "data_100_10.txt", "data_200_10.txt", "data_200_20.txt", "data_400_40.txt", "data_500_50.txt","data_700_70.txt", "data_900_90.txt", "data_1000_100.txt"]

In [39]:
def creat_cost(N, d, t):
    cost_matrix = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(N + 1):
            if i != j:
                cost_matrix[i][j] = d[j] + t[i][j]

    return cost_matrix

In [40]:
def local_search_greedy2(file_data):
    N, K, d, t = read(file_data)
    cost_matrix = creat_cost(N, d, t)

    router = [[0] for i in range(K)]
    cost = [0 for i in range(K)]
    tuple_dis_depot = []
    for i in range(1, N + 1):
        tuple_dis_depot.append((i, cost_matrix[0][i]))
    tuple_dis_depot.sort(key = lambda x: x[1])

    while len(tuple_dis_depot) > 0:
        # lay phan tu dau tien trong tuple_dis_depot
        first = tuple_dis_depot[0][0]
        # xoa phan tu dau tien trong tuple_dis_depot
        tuple_dis_depot.pop(0)
        # Them vao router lam tang it chi phi nhat

        cost_value = [cost[i] + cost_matrix[router[i][-1]][first] for i in range(K)]
        min_cost = min(cost_value)

        index_list = [index for index, value in enumerate(cost_value) if value == min_cost]

        router_selected = index_list[0]
        for index in index_list[1:]:
            if cost[index] < cost[router_selected]:
                router_selected = index
        
        router[router_selected].append(first)
        cost[router_selected] += cost_matrix[router[router_selected][-2]][first]

    for i in range(K):
        router[i].append(0)
        cost[i] += cost_matrix[router[i][-2]][0]
    history_count = 0
    while True:
        max_cost = max(cost)
        max_index = [i for i, j in enumerate(cost) if j == max_cost]
        router_neighborhood = []

        swap1 = np.random.choice(max_index)
        swap2 = np.random.randint(0, K)
        while swap2 == swap1:
            swap2 = np.random.randint(0, K)
        
        if len(router[swap1]) == 3:
            a1 = 1
            b1 = 2
        else:
        #a1 != b1
            a1 = np.random.randint(1, len(router[swap1]) - 2)
            b1 = np.random.randint(a1 + 1, len(router[swap1]) - 1)

        if len(router[swap2]) == 3:
            a2 = 1
            b2 = 2
        else:
            a2 = np.random.randint(1, len(router[swap2]) - 2)
            b2 = np.random.randint(a2 + 1, len(router[swap2]) - 1)

        new_router1 = router[swap1][:a1] + router[swap2][a2:b2] + router[swap1][b1:]
        new_router2 = router[swap2][:a2] + router[swap1][a1:b1] + router[swap2][b2:]

        for i in range(K):
            if i == swap1:
                router_neighborhood.append(new_router1)
            elif i == swap2:
                router_neighborhood.append(new_router2)
            else:
                router_neighborhood.append(router[i])
        
        cost_neighborhood = cal_cost(router_neighborhood, t, d)
        new_max_cost = max(cost_neighborhood)
        if new_max_cost < max_cost:
            router = router_neighborhood
            cost = cost_neighborhood
            history_count = 0
        else:
            history_count += 1
        if history_count == 5000:
            break
        

    print(K)
    for sub in router:
        print(len(sub))
        for pos in sub:
            print(pos, end = ' ')
        print()
    return router, cost, K

In [41]:
def run_algorithm(data_path):
    file_data = "data//" + data_path
    time_start = time.time()
    router, cost, K = local_search_greedy1(file_data)
    time_pro = time.time() - time_start
    result_path = "Local_search_greedy1//4_" + data_path
    with open(result_path, "w") as file:
        file.write("Objective value: " + str(max(cost)) + "\n")
        file.write("Running time: " + str(time_pro) + "\n")
        file.write("Number of vehicles: " + str(K) + "\n")
        file.write("Solution: \n")
        for sub in router:
            for pos in sub:
                file.write(str(pos) + " ")
            file.write("\n")
def run_algorithm2(file_data):
    file_path = "data//" + file_data
    time_start = time.time()
    router, cost, K = local_search_greedy2(file_path)
    time_pro = time.time() - time_start
    result_path = "Local_search_greedy2//4_" + file_data
    with open(result_path, "w") as file:
        file.write("Objective value: " + str(max(cost)) + "\n")
        file.write("Running time: " + str(time_pro) + "\n")
        file.write("Number of vehicles: " + str(K) + "\n")
        file.write("Solution: \n")
        for sub in router:
            for pos in sub:
                file.write(str(pos) + " ")
            file.write("\n")

In [42]:
for data_path in data:
    print(data_path)
    run_algorithm(data_path)
    run_algorithm2(data_path)

data_5_2.txt
2
5
0 4 1 5 0 
4
0 3 2 0 
2
5
0 4 1 2 0 
4
0 3 5 0 
data_5_3.txt
3
4
0 1 3 0 
4
0 4 5 0 
3
0 2 0 
3
4
0 1 3 0 
4
0 4 5 0 
3
0 2 0 
data_10_2.txt
2
7
0 5 8 4 7 9 0 
7
0 2 10 6 1 3 0 
2
7
0 2 5 8 6 9 0 
7
0 10 3 1 4 7 0 
data_10_3.txt
3
5
0 7 5 3 0 
6
0 9 2 8 10 0 
5
0 1 4 6 0 
3
5
0 1 4 3 0 
5
0 8 10 6 0 
6
0 7 2 9 5 0 
data_10_5.txt
5
4
0 7 10 0 
4
0 3 6 0 
4
0 5 2 0 
4
0 9 8 0 
4
0 4 1 0 
5
4
0 5 6 0 
4
0 9 10 0 
4
0 3 2 0 
4
0 7 8 0 
4
0 4 1 0 
data_15_3.txt
3
7
0 10 11 5 3 4 0 
7
0 9 7 1 6 2 0 
7
0 15 8 14 12 13 0 
3
7
0 10 7 8 15 4 0 
7
0 9 11 1 6 3 0 
7
0 14 12 5 2 13 0 
data_15_5.txt
5
5
0 12 15 1 0 
5
0 5 11 4 0 
5
0 10 3 8 0 
6
0 7 9 13 14 0 
4
0 2 6 0 
5
5
0 13 4 8 0 
5
0 15 12 1 0 
6
0 9 3 5 10 0 
4
0 11 6 0 
5
0 7 2 14 0 
data_15_7.txt
7
4
0 8 12 0 
5
0 6 11 2 0 
4
0 9 4 0 
4
0 5 10 0 
4
0 15 13 0 
4
0 3 1 0 
4
0 14 7 0 
7
4
0 8 12 0 
5
0 3 6 10 0 
4
0 9 2 0 
4
0 5 4 0 
4
0 14 7 0 
4
0 15 13 0 
4
0 11 1 0 
data_20_5.txt
5
8
0 10 5 16 13 11 17 0 
5
0 15 12 20 0 
